In [ ]:
"""
Create EOW Services for Web Mapping
"""

data_db  = 'eowdbcontents'
ghost_db = 'flainardbgeoserver'


# GeoServer data
ws = 'eowdeploy'
store = 'ghost'

# Levels table
levels_tbl = 'grid_levels'

# Layers table
layers_tbl = 'layers'

# Reference GRIDS
grid_tbl = 'grids'

gsrvlyr = 'geoserverlayers'

setupdb = 'default'

In [ ]:
import os
import pandas as pd
from shapely.wkt import loads
from glass.geo.gt.sample import create_fishnet
from glass.geo.gt.fmshp import shp_to_obj
from glass.sql.to import df_to_db
from glass.geo.gql.sply import geom_simplification
from glass.sql.i import cols_name
from glass.sql.to import q_to_ntbl
from glass.sql.fm import q_to_obj
from glass.web.geosrv.ws import create_ws
from glass.web.geosrv.stores import create_pgstore
from glass.web.geosrv.lyrs import pub_pglyr
from glass.sql.fm import dump_tbls
from glass.sql.to import restore_tbls
from glass.pyt.oss import del_file
from glass.sql.tbl import drop_tbldata

In [ ]:
import datetime as dt

time_a = dt.datetime.now().replace(microsecond=0)

In [ ]:
# Get Levels
levels = q_to_obj(ghost_db, "SELECT * FROM {}".format(levels_tbl), dbset=setupdb).to_dict(orient='records')

# Get Layers
layers = q_to_obj(data_db, "SELECT * FROM {}".format(layers_tbl), dbset=setupdb).to_dict(orient='records')

In [ ]:
print(levels[0])
print(layers[0])

In [ ]:
from glass.sql.tbl import del_tables

for l in layers:
    del_tables(ghost_db, l["slug"], isBasename=True, db_set=setupdb)

drop_tbldata(data_db, gsrvlyr, dbset=setupdb)

In [ ]:
# Produce layer tables in Ghost DB

# Create Layer Tables
for l in layers:
    q_to_ntbl(ghost_db, l["slug"], l["sql_script"], ntblIsView=False, api='psql', db__set=setupdb)
    
    l["cols"] = cols_name(ghost_db, l["slug"], dbset=setupdb)
    l["cols"].remove(l["geom"])

# Dump and Restore layer tables
"""
tbls_dump = dump_tbls(data_db, list(layers.keys()), os.path.join(
    os.path.dirname(ext_shp), 'viewsdump.sql'
))

restore_tbls(ghost_db, tbls_dump)

# Delete tables from data db
del_tables(data_db, list(layers.keys()))
del_file(tbls_dump)
"""

In [ ]:
Q = (
    "SELECT {c}, ST_SimplifyPreserveTopology("
        "ST_Intersection({g}, ST_GeomFromText('{w}', {ep})), "
        "{sf}"
    ") AS {g} "
    "FROM {t} "
    "WHERE ST_Intersects({g}, ST_GeomFromText('{w}', {ep})){ara}{lmt}"
)

_Q = (
    "SELECT {c}, ST_Intersection({g}, ST_GeomFromText('{w}', {ep})) AS {g} "
    "FROM {t} WHERE ST_Intersects({g}, ST_GeomFromText('{w}', {ep})){ara}{lmt}"
)
    
# For each layer
for l in layers:
    # Publish layers with all data
    pub_pglyr(ws, store, l['slug'], title='tt_' + l['slug'])

maptbl = []
_e = 1
for lvl in levels:
    rgrid = q_to_obj(data_db, "SELECT * FROM {} WHERE level={}".format(
        grid_tbl, str(lvl['id'])
    ), geomCol='geom', epsg=3763, dbset=setupdb)
    
    for idx, row in rgrid.iterrows():
        for l in layers:
            if int(lvl['id']) not in l['levels']:
                continue
            
            # Clip and simplification
            if pd.isna(lvl['simplification']) or l["simplification"] == False:
                nt = q_to_ntbl(
                        ghost_db, '{}_lvl{}_id{}'.format(
                            l["slug"], str(lvl['id']), str(row.id)
                        ), _Q.format(
                            c=", ".join(l["cols"]), g=l["geom"],
                            w=row.geom.wkt, ep='3763', t=l["slug"],
                            lmt="" if pd.isna(lvl['nfeatures']) else " LIMIT {}".format(str(int(lvl['nfeatures']))),
                            ara=" AND ST_Area({}) >= {}".format(
                                l["geom"], lvl['minarea']
                            ) if l['tgeom'] == 'Polygon' and not pd.isna(lvl['minarea']) else ""
                        ),
                        api='psql', db__set=setupdb
                )
            else:
                nt = q_to_ntbl(
                        ghost_db, '{}_lvl{}_id{}'.format(
                            l["slug"], str(lvl['id']), str(row.id)
                        ), Q.format(
                            c=", ".join(l["cols"]), g=l["geom"], w=row.geom.wkt,
                            ep='3763', t=l["slug"], sf=str(lvl['simplification']),
                            lmt="" if pd.isna(lvl["nfeatures"]) else " LIMIT {}".format(str(int(lvl['nfeatures']))),
                            ara=" AND ST_Area({}) >= {}".format(
                                l["geom"], lvl['minarea']
                            ) if l['tgeom'] == 'Polygon' and not pd.isna(lvl['minarea']) else ""
                        ),
                        api='psql', db__set=setupdb
                )
            
            # Publish layer in GeoServer
            pub_pglyr(ws, store, nt, title='tt_' + nt)
            
            maptbl.append(
                [_e, nt, l['id'], row.id]
            )
            
            _e += 1

In [ ]:
#maptbl = pd.DataFrame(
    #maptbl, columns=['id', 'name', 'layerid', 'cellid']
#)
df_to_db(data_db, maptbl, gsrvlyr, append=True, dbset=setupdb)

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
data_db  = 'eow-db-contents'
ghost_db = 'flainar-db-geoserver'


# GeoServer data
ws = 'eowdeploy'
store = 'ghost'

gsrvlyr = 'geoserverlayers'

setupdb = 'flainar'

In [ ]:
from glass.sql.fm import q_to_obj
from glass.web.geosrv.ws import create_ws
from glass.web.geosrv.stores import create_pgstore
from glass.web.geosrv.lyrs import pub_pglyr

In [ ]:
glyr = q_to_obj(data_db, "SELECT name FROM {}".format(gsrvlyr), dbset=setupdb)

In [ ]:
glyr.head()

In [ ]:
# Create workspace and store in GeoServer

create_ws(ws, overwrite=True)

create_pgstore(store, ws, ghost_db)

In [ ]:
# Create layers in geoserer
for idx, row in glyr.iterrows():
    pub_pglyr(ws, store, row['name'], title='tt_' + row['name'])